In [15]:
import pandas as pd
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder

In [2]:
data= pd.read_csv('healthcare-dataset-stroke-data.csv')


In [3]:
data.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [4]:
data.isnull().sum()

id                     0
gender                 0
age                    0
hypertension           0
heart_disease          0
ever_married           0
work_type              0
Residence_type         0
avg_glucose_level      0
bmi                  201
smoking_status         0
stroke                 0
dtype: int64

In [5]:
import math
bmi_mean=data['bmi'].mean()
bmi_mean=math.ceil(bmi_mean)
bmi_mean

29

In [6]:
data['bmi']=data.bmi.fillna(bmi_mean)

In [7]:
data.isnull().sum()

id                   0
gender               0
age                  0
hypertension         0
heart_disease        0
ever_married         0
work_type            0
Residence_type       0
avg_glucose_level    0
bmi                  0
smoking_status       0
stroke               0
dtype: int64

In [8]:
gen=LabelEncoder()
data['gender']=gen.fit_transform(data.gender)

married=LabelEncoder()
data['ever_married']=married.fit_transform(data['ever_married'])

work=LabelEncoder()
data['work_type']=work.fit_transform(data['work_type'])

residence=LabelEncoder()
data['Residence_type']=residence.fit_transform(data['Residence_type'])

smoking=LabelEncoder()
data['smoking_status']=smoking.fit_transform(data['smoking_status'])


In [9]:
data

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,1,67.0,0,1,1,2,1,228.69,36.6,1,1
1,51676,0,61.0,0,0,1,3,0,202.21,29.0,2,1
2,31112,1,80.0,0,1,1,2,0,105.92,32.5,2,1
3,60182,0,49.0,0,0,1,2,1,171.23,34.4,3,1
4,1665,0,79.0,1,0,1,3,0,174.12,24.0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5105,18234,0,80.0,1,0,1,2,1,83.75,29.0,2,0
5106,44873,0,81.0,0,0,1,3,1,125.20,40.0,2,0
5107,19723,0,35.0,0,0,1,3,0,82.99,30.6,2,0
5108,37544,1,51.0,0,0,1,2,0,166.29,25.6,1,0


In [10]:
data=data.drop('id',axis=1)

In [11]:
x=data.drop("stroke",axis=1)
y=data['stroke']

In [12]:
model_params = {'knn': {'model':KNeighborsClassifier(),'params':{'n_neighbors':[8,12,20]}},
                'DT': {'model':DecisionTreeClassifier(),'params':{'max_depth':[6,10,14]}},
                'svm': {'model':SVC(gamma='auto'),'params':{'C':[1,10,20],'kernel':['rbf','linear']}},
               'RF':{'model':RandomForestClassifier(),'params':{'n_estimators':[5,10,15]}}}

In [16]:
scores =[]
models =['knn','DT','svm','RF']
for model_name in models:
    mp= model_params[model_name]
    gds=RandomizedSearchCV(mp['model'],mp['params'], cv=5, return_train_score=True)
    gds.fit(x,y)
    scores.append({'model': model_name, 'best score:' :gds.best_score_,'best_params:': gds.best_params_})

In [14]:
scores

[{'model': 'knn',
  'best score:': 0.9512720156555773,
  'best_params:': {'n_neighbors': 20}},
 {'model': 'DT',
  'best score:': 0.9444227005870841,
  'best_params:': {'max_depth': 6}},
 {'model': 'RF',
  'best score:': 0.947358121330724,
  'best_params:': {'n_estimators': 10}},
 {'model': 'svm',
  'best score:': 0.9512720156555773,
  'best_params:': {'kernel': 'linear', 'C': 1}}]